In [6]:
import pandas as pd
import json 
import pickle
from pathlib import Path

In [7]:
files = sorted(Path('.\data\characters').glob('*.json'))

names = []
species = []
paragraphs = []
neighbours = []
gender = []

data_dict = {}
for fn in files:
    with open(fn, 'rb') as f:
        part = json.load(f)
        names.append(part['name'])
        
        if 'content' in part:
            paragraphs.append(part['content'])
        else:
            paragraphs.append(None)

        if 'crosslinks' in part:
            crosslinks = []
            for link in part['crosslinks']:
                crosslinks.append(link.split('/')[-1])
            neighbours.append(crosslinks)
        else:
            crosslinks.append(None)

        if 'side_bars' in part:
            found_phys_desc = False
            for i in part['side_bars']:
                if i['name'] == 'Physical description':
                    found_phys_desc = True

                    found_species = False
                    found_gender = False
                    for att in i['attributes']:
                        if att['name'] == 'Species' and att['values'] is not None:
                            species.append(att['values'][0])
                            found_species = True                            
                        if att['name'] == 'Gender' and att['values'] is not None:
                            gender.append(att['values'][0])
                            found_gender = True
                    if not found_species: species.append(None)
                    if not found_gender: gender.append(None)
            if not found_phys_desc:
                species.append(None)
                gender.append(None)
        else:
            species.append(None)
            gender.append(None)

In [8]:
data = pd.DataFrame(columns = ['Name', 'Paragraph', 'Neighbours', 'Species', 'Gender'])
data['Name'] = names
data['Paragraph'] = paragraphs
data['Neighbours'] = neighbours
data['Species'] = species
data['Gender'] = gender

In [9]:
data.head(5)

,Name,Paragraph,Neighbours,Species,Gender
0,"""Backdraft""","""Backdraft"" was the nickname of a male human T...","[Canon, Human, Gender, Color, First_Order, Fir...",Human,Male
1,"""Blue"" Imcrix","""Blue"" Imcrix was an infamous male Taloron cri...","[Star_Wars_Legends, Taloron, Legends, Legends,...",Taloron,Male
2,"""Blue Five"" (Batuu)","""Blue Five"" was a female human pilot that flew...","[Canon, Batuu_system, First_Order-Resistance_W...",Human,Female
3,"""Frog Lady""","""Frog Lady"" was the nickname of an amphibious ...","[Canon, Category:Articles_to_be_expanded, Talk...",Unidentified frog-like anthropoid,Female
4,"""Frog Man""","""Frog Man"" was the husband of Frog Lady who li...","[Canon, Unidentified_frog-like_anthropoid, Gen...",Unidentified frog-like anthropoid,Male


In [10]:
with open('./data/data_all.pickle', 'wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)